# 함수형 API로 복잡한 신경망 만들기

In [1]:
import tensorflow as tf
import numpy as np

/Users/barrett11357/coding/pycharm/handson_ml_--/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

datasets=fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(datasets.data, datasets.target, random_state=42)#훈련세트와 테스트세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)#훈련세트에서 검증세트 분리

## 와이드 & 딥 신경망 구현

입력의 일부 또는 전체가 출력 층에 바로 연결되는 와이드 & 딥 신경망을 구현해볼거다.
이 구조를 사용하면 깊게 쌓인 층을 이용해 복잡한 패턴과 짧은 경로를 사용한 간단한 규칙을 모두 학습할 수 있다.

함수형 API를 사용하면 이런 순차적이지 않은 신경망을 구현할 수 있다.

### 모든 특성을 짧은 경로로 연결

In [4]:
#먼저 필요한 레이어들 준비한다.
norm_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(50, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(50, activation="relu")
concat_layer = tf.keras.layers.Concatenate()#얘가 여러 입력 텐서를 하나의 입력 텐서로 연결시키는 역할을 한다. 
output_layer = tf.keras.layers.Dense(1)

#이 밑 부분부터가 특이하다. 각각의 레이어들을 함수처럼 사용한다.
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
nomalized = norm_layer(input_)
hidden1 = hidden_layer1(nomalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([nomalized, hidden2])#concat layer를 사용해서 입력과 두번째 은닉층의 출력을 연결시킨다.
output = output_layer(concat)

model = tf.keras.Model(inputs=[input_], outputs=[output])

2025-02-02 20:30:49.637752: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-02-02 20:30:49.637781: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-02 20:30:49.637796: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-02 20:30:49.637831: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-02 20:30:49.637842: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 일부 특성만 짧은 경로로 연결

이 경우에는 입력을 여러개 사용하면 된다.

그리고 이번에는 Dense층의 생성과 호출을 동시에 해보면서 코드를 좀 더 간략하게 짜 볼 것이다.
Nomalization층은 fit()을 호출하기 전에 adapt()를 호출해서 데이터에 적응시켜야 하기 때문에 참조값을 따로 유지해야 해서 이런 식으로 쓸 수는 없다.

In [5]:
#짧은 경로
input_wide = tf.keras.layers.Input(shape=[5])#특성인덱스 0부터 4까지, 오해하면 안되는게 단순히 입력의 크기가 5라는 얘기다. 슬라이싱 해서 입력하는 것은 모델 밖에서 하게 되는 일이다.
#깊은 경로
input_deep = tf.keras.layers.Input(shape=[6])#특성인덱스 2부터 7까지

#Nomalization층은 fit()을 호출하기 전에 adapt()를 호출해서 데이터에 적응시켜야 하기 때문에 참조값을 따로 유지해야 한다.
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()


norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(50, activation="relu")(hidden1)

concat = tf.keras.layers.Concatenate()([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)


model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

좀 더 부연설명을 하자면 norm_wide, hidden1, concat, output 같은 변수들은 지금 레이어를 담고 있는게 아니다.
레이어 객체 자체가 아니라, 해당 레이어를 입력 텐서에 적용한 후 생성된 출력 텐서를 담고 있는 변수이다. 

예를 들어 `hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)` 이 코드에서
`tf.keras.layers.Dense(50, activation="relu")` 이 코드로 Dense 객체를 생성하고, 
생성된 Dense 레이어 객체를 norm_deep 텐서에 적용하여 출력 텐서를 계산하고, 그 결과를 hidden1에 저장한 것이다. 

컴파일은 앞에서 했던 것과 똑같이 하면 되고,

모델을 훈련할 때 하나의 입력 행렬 X_train을 넣으면 되는게 아니고
입력 마다 하나씩 행렬의 튜플 (X_train_wide, X_train_deep)을 넣어줘야 한다. 
검증과 테스트 데이터에도 마찬가지이다. 

In [6]:
model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=[tf.keras.metrics.RootMeanSquaredError()])


In [7]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=1, validation_data=((X_valid_wide, X_valid_deep), y_valid))

2025-02-02 20:30:50.433508: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 2.7706 - root_mean_squared_error: 1.6230 - val_loss: 2.3664 - val_root_mean_squared_error: 1.5383


### 여러개의 출력 사용하기

보조출력을 만드는 것은 쉬워용. 그냥 출력층을 하나 만들고, Model객체를 만들 때 outputs리스트에 그 출력층을 추가해주기만 하면 된다.

In [8]:
#위의 일부특성만 짧은 경로로 연결하는 코드를 그대로 가져옴.
input_wide = tf.keras.layers.Input(shape=[5])#특성인덱스 0부터 4까지, 오해하면 안되는게 단순히 입력의 크기가 5라는 얘기다. 슬라이싱 해서 입력하는 것은 모델 밖에서 하게 되는 일이다.
input_deep = tf.keras.layers.Input(shape=[6])#특성인덱스 2부터 7까지

#Nomalization층은 fit()을 호출하기 전에 adapt()를 호출해서 데이터에 적응시켜야 하기 때문에 참조값을 따로 유지해야 한다.
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(50, activation="relu")(hidden1)

concat = tf.keras.layers.Concatenate()([norm_wide, hidden2])

output = tf.keras.layers.Dense(1, name="output")(concat)#출력 층에 따로 이름 붙여줌.
aux_output = tf.keras.layers.Dense(1, name="aux_output")(hidden2)#새로 추가한 보조출력

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

각 출력은 자신만의 손실함수를 가져야 한다.
컴파일 할 때 손실의 리스트를 전달해주면 된다.
기본적으로 케라스는 나열된 손실을 모두 더하여 **최종**손실을 구해 훈련에 사용한다고 한다.

In [9]:
model.compile(loss=["mse","mse"], 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              metrics=["RootMeanSquaredError", "RootMeanSquaredError"])#출력이 2개 이므로 메트릭스도 2개를 넣어줘야 한다.

이떄 손실함수를 위와같이 리스트 말고, 출력이름-손실함수 쌍의 딕셔너리로 전달할 수도 있다.

In [10]:
model.compile(loss={"output":"mse", "aux_output":"mse"}, 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              metrics=["RootMeanSquaredError", "RootMeanSquaredError"])


In [11]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_4     │ (None, 6)         │         13 │ input_layer_4[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 50)        │        350 │ normalization_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_3     │ (None, 5)         │         11 │ input_layer_3[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 50)        │      2,550 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 55)        │          0 │ normalization_3[… │
│ (Concatenate)       │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         56 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ aux_output (Dense)  │ (None, 1)         │         51 │ dense_7[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,031 (11.85 KB)

 Trainable params: 3,007 (11.75 KB)

 Non-trainable params: 24 (104.00 B)

예측을 수행할 때는 출력이 여러개이기 때문에 각 출력에 대해서 따로따로 레이블을 제공해야 한다. 
다만 지금 코드에서는 주출력과 보조출력이 같은 것을 예측하므로 동일한 레이블을 사용한다. 

여기에서도 그냥 튜플로 전달하거나, 출력이름과 매칭한 딕셔너리로 전달할 수도 있다.

In [12]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

model.fit((X_train_wide, X_train_deep),(y_train, y_train),
          epochs=1,
          validation_data=((X_valid_wide, X_valid_deep),(y_valid, y_valid)))

363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - aux_output_RootMeanSquaredError: 1.8061 - aux_output_loss: 3.3281 - loss: 6.4813 - output_RootMeanSquaredError: 1.7583 - output_loss: 3.1532 - val_aux_output_RootMeanSquaredError: 8.5328 - val_aux_output_loss: 72.7719 - val_loss: 141.3507 - val_output_RootMeanSquaredError: 8.2790 - val_output_loss: 68.5063


In [16]:
y_pred_main, y_pred_aux = model.predict((X_train_wide[:3], X_train_deep[:3]))
print(y_pred_main)
print(y_pred_aux)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[2.0454748]
 [2.1194348]
 [2.4531488]]
[[2.4120953]
 [2.2908456]
 [2.5195816]]


# 서브클래싱 API로 동적 모델 만들기

시퀀셜 API와 함수형 API는 모두 선언적이다.
사용할 층과 연결방식을 먼저 정의해야 한다. 그 다음에 모델에 데이터를 주입해서 훈련이나 추론을 시작할 수 있다.
이 방식은 장점이 많음. 모델을 저장하거나 공유하기 쉽고, 모델의 구조를 출력하거나 분석하기 쉽다. 전체 모델이 층으로 구성된 정적 그래프이므로 디버깅 하기도 쉽다.

하지만 **정적이라는 것이 단점이 되기도 한다.**
어떤 모델은 반복문을 포함하고, 다양한 크기를 다루어야 하며 조건문을 가지는 등 여러가지 동적인 구조를 필요로 한다.
이런 경우 조금 더 명령형 프로그래밍 스타일이 필요하면 **서브클래싱 API**가 정답이다.

간단하게 Model클래스를 상속하는 클래스를 만든다.
생성자 함수 안에서 필요한 층을 만든다.
그 다음 call()메서드 안에서 수행하려는 연산을 기술한다.

바로 위에서 함수형 API로 구현한 와이드 & 딥 보조출력 모델을 서브클래싱 API로 작성해보자.

In [35]:
# Custom class 등록: @register_keras_serializable 데코레이터 추가
@tf.keras.utils.register_keras_serializable(package='Custom')
class WideAndDeep(tf.keras.Model):
    #생성자 함수에서 필요한 층을 모두 정의한다.
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden_layer1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden_layer2 = tf.keras.layers.Dense(units, activation=activation)
        self.output_layer = tf.keras.layers.Dense(1)
        self.aux_output_layer = tf.keras.layers.Dense(1)

    # `call()` 메서드는 Tensorflow의 **서브클래싱 API**로 사용자 정의 모델을 작성할 때, 모델의 Forward Propagation(순전파)을 정의하는 핵심 메서드입니다.
    # 하지만 이 메서드는 보통 사용자가 직접 호출하기보다는, Keras 내부적으로 **모델 훈련, 평가, 예측 작업 중 자동으로 호출**됩니다.
    def call(self, inputs): #별도의 Input레이어를 따로 만들 필요 없이 call 함수의 매개변수 inputs를 사용하면 된다고 한다.
        input_wide, input_deep = inputs #입력이 나뉘어져서 튜플 형태로 들어올 것이다.

        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden_layer1(norm_deep)
        hidden2 = self.hidden_layer2(hidden1)
        concat = tf.keras.layers.Concatenate()([norm_wide, hidden2])
        output = self.output_layer(concat)
        aux_output = self.aux_output_layer(hidden1)

        return output, aux_output


In [36]:
    model = WideAndDeep(units=30, activation="relu", name="my_cool_model")
    model.summary()

Model: "my_cool_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_9 (Normalization) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_10                │ ?                      │   0 (unbuilt) │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

이제 모델 객체가 생겼으니까 이전에 하던 것처럼 모델 객체를 컴파일하고, 정규화 층을 적응시키고, 훈련하고 평가하면 된다.

이 API의 가장 큰 차이점은 call()메서드 안에 어떤 코드던지 넣을 수 있다는 것이다.

그런데 이러한 **유연성에는 대가가 따른다.** 모델의 구조가 call()메서드 안에 숨겨져 있어서 케라스가 쉽게 검사할 수 없고, tf.keras.clone_model()을 사용해서 모델을 복사할 수 없다.

따라서 추가적인 유연성이 정말 필요한 경우가 아니라면 그냥 시퀀셜이나 함수형 API를 사용하는 것이 좋다고 한다.

# 모델 저장하고 로드하기

먼저 간단하게 컴파일하고 훈련한다. 그래야 저장될 가중치가 있음.

In [37]:
model.compile(loss=["mse","mse"], optimizer="Adam", metrics=["RootMeanSquaredError", "RootMeanSquaredError"])

In [38]:
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

model.fit((X_train_wide, X_train_deep),(y_train, y_train),
          epochs=1,
          validation_data=((X_valid_wide, X_valid_deep),(y_valid, y_valid)))

363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - RootMeanSquaredError: 1.7785 - RootMeanSquaredError_1: 2.1267 - loss: 7.8368 - mse_loss: 4.5693 - val_RootMeanSquaredError: 2.6912 - val_RootMeanSquaredError_1: 8.1389 - val_loss: 73.4841 - val_mse_loss: 66.2076


.keras파일의 구조에 대한 자세한 정보는 책을 참고하자.

In [39]:
#model.save("my_model", save_format="tf") #<-책에서는 이런 식으로 하라고 하는데 막상 실행해보면 deprecated라고 경고뜸.# 수정된 코드
model.save("my_model.keras") #이렇게 해야한다. 좀 지나면 my_model.keras파일이 나타난다.

이제 모델을 불러와본다.

In [40]:
# .keras 파일 로드
model = tf.keras.models.load_model("my_model.keras")

# 모델의 구조 출력
model.summary()

'''
처음에는 WideAndDeep클래스에 대한 정보를 불러올 수 없다면서 오류가 발생했었다.
클래스에 @tf.keras.utils.register_keras_serializable(package='Custom') 데코레이터를 붙여서 커스텀클래스로 등록하니까 해결됨.
'''

Model: "my_cool_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_11                │ (None, 5)              │            11 │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ normalization_12                │ (None, 6)              │            13 │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 30)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,647 (14.26 KB)

 Trainable params: 1,207 (4.71 KB)

 Non-trainable params: 24 (104.00 B)

 Optimizer params: 2,416 (9.44 KB)

In [41]:
#불러온 모델을 마저 더 훈련시킬 수도 있고, 예측을 생성할 수도 있다. 파라미터 값만 저장하고 로드할 수도 있다.
model.predict((X_train_wide[:3], X_train_deep[:3]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


(array([[1.8094933],
        [2.3374226],
        [2.4998925]], dtype=float32),
 array([[1.9561379],
        [1.698771 ],
        [1.6316702]], dtype=float32))

# 콜백 사용하기

콜백, 말그대로 콜백 함수다.
fit()메서드의 callbacks매개변수를 사용해서 케라스가 훈련의 시작 전이나 후에 호출할 수 있는 객체 리스트를 전달할 수 있다. 또는 에포크의 시작 전후, 각 배치 처리 전후에 호출할 수 있다.

예를 들어서 `ModelCheckPoint`는 훈련하는 동안 일정한 간격으로 모델의 체크포인트를 저장한다. 아래 코드로 가중치만 저장하게 한다.

In [50]:
'''
**`my_checkpoint.weights.h5` 파일에는 에포크별 가중치가 저장되지 않습니다.** 대신, **최신 에포크의 가중치만** 저장됩니다.

### 파일에 각 에포크의 가중치를 저장하려면?
각 에포크의 가중치를 별도 파일로 저장하려면, `filepath`에서 에포크 번호를 포함하도록 설정해야 합니다.
'''
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "my_checkpoint.weights.h5",
    save_weights_only=True
)

# 가정된 모델 및 학습 작업
model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train),
    epochs=1,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[checkpoint_cb]
)

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - RootMeanSquaredError: 0.9940 - RootMeanSquaredError_1: 0.9313 - loss: 1.8712 - mse_loss: 0.8691 - val_RootMeanSquaredError: 4.0104 - val_RootMeanSquaredError_1: 2.1300 - val_loss: 20.6203 - val_mse_loss: 4.5349


그리고 훈련하는 동안 검증세트를 사용하면 ModelCheckpoint를 만들 떄 save_best_only=True로 설정할 수 있는데
이렇게 하면 최상의 검증 세트 점수에서만 모델을 저장한다고 한다.
모델이 훈련세트에 과적합될 걱정을 하지 않아도 된다고 한다.
이는 **조기종료**기법을 구현하는 한 가지 방법이다. 단 최상의 모델 가중치를 저장할 뿐이지, 훈련을 멈추지는 않음.

또 다른 방법은 `EarlyStopping`콜백을 사용하는 것이다.
일정 에포크동안 검증세트에 대한 점수가 향상되지 않으면 훈련을 종료한다.
restore_best_weight=True로 지정하면 훈련이 끝난 후 최상의 가중치로 모델을 복원한다.

이 두가지 콜백을 함께 사용하는 것이 좋을 수 있음.

In [51]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=3, restore_best_weights=True
)

model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train),
    epochs=1,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[checkpoint_cb, early_stopping_cb]#체크포인트저장 콜백과 조기종료 콜백을 함께 사용
)

363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - RootMeanSquaredError: 0.7432 - RootMeanSquaredError_1: 0.9010 - loss: 1.3754 - mse_loss: 0.8122 - val_RootMeanSquaredError: 28.6246 - val_RootMeanSquaredError_1: 1.1608 - val_loss: 820.7138 - val_mse_loss: 1.3470


더 많은 제어를 원한다면 사용자 정의 콜백을 만들 수도 있다.
예를 들어서 아래의 콜백은 훈련하는 동안 검증손실과 훈련손실의 비율을 출력한다.

In [53]:
class PrintValTrainRatioCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs["val_loss"]
        train_loss = logs["loss"]
        val_ratio = val_loss / train_loss
        print(f"Epoch {epoch}: val_loss / train_loss = {val_ratio:.4f}")

model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train),
    epochs=1,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[checkpoint_cb, early_stopping_cb, PrintValTrainRatioCallback()]
)

359/363 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - RootMeanSquaredError: 1.4068 - RootMeanSquaredError_1: 0.9517 - loss: 2.9466 - mse_loss: 0.9079Epoch 0: val_loss / train_loss = 4.6472
363/363 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - RootMeanSquaredError: 1.4039 - RootMeanSquaredError_1: 0.9512 - loss: 2.9372 - mse_loss: 0.9069 - val_RootMeanSquaredError: 2.8022 - val_RootMeanSquaredError_1: 1.6242 - val_loss: 10.4905 - val_mse_loss: 2.6370


딱 보면 알겠지만 `tf.keras.callbacks.Callback`를 상속해서 각종 메서드들을 구현하면 되는 것이다.
아래는 `tf.keras.callbacks.Callback`에서 재정의할 수 있는 메서드들의 목록입니다:
- `on_train_begin`
- `on_train_end`
- `on_epoch_begin`
- `on_epoch_end`
- `on_train_batch_begin`
- `on_train_batch_end`
- `on_test_begin`
- `on_test_end`
- `on_test_batch_begin`
- `on_test_batch_end`
- `on_predict_begin`
- `on_predict_end`
- `on_predict_batch_begin`
- `on_predict_batch_end`

콜백은 훈련 단계 뿐만 아니라 검증과 예측 단계에서도 사용할 수 있다.

# 텐서보드로 시각화하기

텐서보드를 사용하려면 프로그램(여기에서는 모델)을 변경해서 `이벤트 파일`이라는 특별한 이진 로그 파일에 시각화하려는 데이터를 출력해야 한다. 
그러면 텐서보드가 로그 디렉터리를 모니터링하고 자동으로 변경사항을 읽어서 그래프를 업데이트한다. 
훈련하는 중간에 학습곡선 같은 실시간 데이터를 시각화할 수 있다. 

일반적으로 텐서보드가 로그디렉터리를 가리키고, 프로그램은 실행할 때마다 다른 서브디렉터리에 이름을 업데이트한다. 
이렇게 하면 복잡하지 않게 하나의 텐서보드 서버가 여러 번 실행한 프로그램의 결과를 시각화하고 비교할 수 있다. 

그래서 일단 루트디렉터리 이름을 my_logs로 지정하고,
호출할 떄마다 현재 날짜와 시간을 이름으로 하는 my_logs의 서브디렉터리 경로를 생성하는 함수를 하나 만들겠다.

In [55]:
from pathlib import Path
from time import strftime

def get_log_dir():
    root_logdir = Path("my_logs")
    logdir = root_logdir / strftime("run_%Y_%m_%d-%H_%M_%S")
    return logdir

run_logdir = get_log_dir()
print(run_logdir)

my_logs/run_2025_02_02-23_36_35


케라스가 로그디렉터리를 생성하고, 훈련하는 중에 이벤트 파일을 만들어서 요약 정보를 기록하는 편리한 `TensorBoard`콜백을 이미 만들어두었다!!!
모델의 훈련 및 검증 손실과 측정지표를 계산하고 신경망의 프로파일링도 수행한다.

In [57]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100,200))#신경망이 워밍업 하는데 몇 개의 배치가 걸리는 경우가 있으므로 너무 일찍 프로파일링 하지 않는 게 좋다고 한다. 또 프로파일링에는 자원이 소모되므로 모든 배치에 대해서 프로파일링을 수행하지 않는 것이 좋다고 한다.

2025-02-02 23:40:50.389446: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-02 23:40:50.389453: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2025-02-02 23:40:50.391216: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


In [58]:
model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train),
    epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
    callbacks=[tensorboard_cb]
)

Epoch 1/10
120/363 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - RootMeanSquaredError: 0.9700 - RootMeanSquaredError_1: 0.9064 - loss: 1.9175 - mse_loss: 0.8248

2025-02-02 23:41:00.181239: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-02 23:41:00.181249: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


212/363 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - RootMeanSquaredError: 1.2149 - RootMeanSquaredError_1: 0.9315 - loss: 2.5120 - mse_loss: 0.8703

2025-02-02 23:41:01.150947: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-02 23:41:01.192723: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-02 23:41:01.194988: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: my_logs/run_2025_02_02-23_36_35/train/plugins/profile/2025_02_02_23_41_01/Mac-mini.local.xplane.pb


363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - RootMeanSquaredError: 1.2452 - RootMeanSquaredError_1: 0.9365 - loss: 2.5288 - mse_loss: 0.8786 - val_RootMeanSquaredError: 3.1830 - val_RootMeanSquaredError_1: 1.4400 - val_loss: 12.2047 - val_mse_loss: 2.0728
Epoch 2/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - RootMeanSquaredError: 1.5394 - RootMeanSquaredError_1: 0.9537 - loss: 3.4369 - mse_loss: 0.9102 - val_RootMeanSquaredError: 3.9390 - val_RootMeanSquaredError_1: 2.0108 - val_loss: 19.5588 - val_mse_loss: 4.0416
Epoch 3/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - RootMeanSquaredError: 1.2098 - RootMeanSquaredError_1: 0.9280 - loss: 2.7157 - mse_loss: 0.8633 - val_RootMeanSquaredError: 0.9110 - val_RootMeanSquaredError_1: 2.2806 - val_loss: 6.0312 - val_mse_loss: 5.1990
Epoch 4/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - RootMeanSquaredError: 1.0533 - RootMeanSquaredError_1: 0.9203 - loss: 2.0277 - mse_loss: 0.8471 - val_RootMeanSquaredError: 1.2219 - val_RootMeanSquaredError_1

그리고 텐서보드 서버를 실행한다! 뭐가 막 뜨는데 신기함!!!
localhost:6006으로 접속하면 된다!

이게 종료시킬 때는 어떻게 해야하는지 모르겠는데 나는 일단 터미널에서 6006포트를 사용하는 어플리케이션을 찾아서 종료시키는 식으로 함.
`lsof -i :6006`으로 6006포트를 사용하는 어플리케이션의 PID를 알아내고,
`kill -9 PID번호`로 프로세스를 종료함.



In [60]:
%load_ext tensorboard
%tensorboard --logdir=my_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# 신경망 하이퍼파라미터 튜닝하기

다층 퍼셉트론의 층 수, 각 층의 뉴런 수, 각 층에서 사용하는 활성화함수, 가중치 초기화 전략, 옵티마이저, 학습률, 배치 크기 등 많은 것을 바꿀 수 있다.

이를 조정하기 위해서 **keras-tuner**라이브러리를 사용한다.

## 모델 하이퍼파라미터 튜닝

이제 밑에서 build_model()이라는 함수를 만들건데,

우선 이 함수는 매개변수로 keras-tuner라이브러리의 HyperParameters객체를 받는다.
HyperParameters객체를 사용해서 튜닝 가능한 하이퍼파라미터를 정의하고, 가능한 값의 범위를 설정할 수 있다. 그리고 이러한 하이퍼파라미터를 사용해서 모델을 만들고 컴파일 할 수 있다.

함수의 윗부분에서는 필요한 하이퍼파라미터들을 정의하고,
아랫부분에서는 이를 사용해서 모델을 만들어 반환하는 코드를 작성할 것이다.

In [88]:
import keras_tuner as kt

norm_layer = tf.keras.layers.Normalization()

def build_model(hp): #HyperParameters객체를 매개변수로 받는다. 이 매개변수는 뒤에서 kt.Randomsearch 튜너가 넣어줄 것이다.

    #"n_hidden"이라는 새로운 하이퍼파라미터가 hp에 있는지 확인하고, 있다면 그 값을 반환한다.
    #그렇지 않은 경우 가능한 값의 범위가 0에서 8인 "n_hidden"이라는 새로운 정수 하이퍼파라미터를 등록하고 기본값 2를 반환한다.
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    #optimizer는 adam과 sgd 둘 중 하나로 선택된다. 일단 문자열로 저장되고, 아래 조건문에서 실제 옵티마이저 객체를 넣어주는 것이다.
    optimizer = hp.Choice("optimizer", values=["adam", "sgd"])
    if optimizer == "adam" :
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    #이제 하이퍼파라미터를 사용해서 모델을 구축하면 된다.

    model = tf.keras.Sequential() #시퀀셜 모델 생성
    model.add(norm_layer)  # 정규화 층 추가
    model.add(tf.keras.layers.Flatten())
    for i in range(n_hidden): #은닉층 추가
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))

    #모델을 컴파일해서 반환한다. 옵티마이저도 위에서 정의한 하이퍼파라미터 변수를 가져다가 쓴다.
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

이제 기본적인 **랜덤서치**를 실행한다.

kt.RandomSearch 튜너를 사용한다.

튜너를 만들고, build_model 함수를 생성자에 전달한 후 튜너의 search()메서드를 호출하면 된다.

In [89]:
'''
튜너는 먼저 모든 하이퍼파라미터 사양을 수집하기 위해 빈 HyperParameters객체로 build_model()을 호출한다. 그러면 hp객체에 하이퍼파라미터들이 등록됨.
그런 다음에 이 코드에서는 5번 시도한다.
각 시도마다 하이퍼파라미터 범위 내에서 랜덤하게 샘플링된 하이퍼파라미터를 사용해서 모델을 만든 다음에 해당 모델을 10 에포크동안 훈련하고, 각 체크포인트를 my_dir/my_model 디렉터리에 저장한다.
'''
random_search_tuner = kt.RandomSearch(
    build_model,
    objective="val_root_mean_squared_error",#이 설정으로 튜너는 RMSE가 낮은 모델을 선호한다.
    max_trials=5,
    directory="my_dir", #디렉터리와 프로젝트이름은 튜닝결과를 저장하는데 사용한다.
    project_name="my_model",
    overwrite=True
)

norm_layer.adapt(X_train)

random_search_tuner.search(
    X_train,
    y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    verbose=2
)

Trial 5 Complete [00h 00m 19s]
val_root_mean_squared_error: 1.0564414262771606

Best val_root_mean_squared_error So Far: 0.7217630743980408
Total elapsed time: 00h 01m 26s


adam 옵티마이저가 확실히 수렴 속도도 빠르고, sgd보다 훨씬 안정적이네. 근데 내가 뭔가 코드를 잘못 짰는지 rmse가 너무 크게 나오는데.... 학습률이 너무 높은 것 같기도 하고....? 아니다. 학습률은 문제 없는데....

아래 코드로 최상의 하이퍼파라미터 값을 얻는다.

In [105]:
top_3_models = random_search_tuner.get_best_hyperparameters(num_trials=3)
for i, hp in enumerate(top_3_models):
    print(f"Top {i+1} Model Hyperparameters:")
    print(hp.values)#하이퍼파라미터 값 출력

Top 1 Model Hyperparameters:
{'n_hidden': 1, 'n_neurons': 156, 'learning_rate': 0.0024650435245075655, 'optimizer': 'sgd'}
Top 2 Model Hyperparameters:
{'n_hidden': 6, 'n_neurons': 191, 'learning_rate': 0.0001752763052189826, 'optimizer': 'sgd'}
Top 3 Model Hyperparameters:
{'n_hidden': 0, 'n_neurons': 175, 'learning_rate': 0.005118522483814269, 'optimizer': 'sgd'}


아래 코드로 최상의 모델을 얻는다.

In [107]:
best_model = random_search_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 8)              │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 156)            │         1,404 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           157 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,578 (6.17 KB)

 Trainable params: 1,561 (6.10 KB)

 Non-trainable params: 17 (72.00 B)

각 튜너는 소위 **오라클**의 안내를 받는데 튜너는 각 시도 전에 오라클에 다음 시도가 무엇인지 알려달라고 요청한다.

랜덤 설치 튜너는 다음 시도를 랜덤으로 선택하는 아주 기본적인 RandomSearchOracle을 사용한다.
오라클은 모든 시도를 기록하기 때문에 최상의 시도를 요청해서 해당 시도의 요약을 출력할 수 있다.

In [94]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 1 summary
Hyperparameters:
n_hidden: 1
n_neurons: 156
learning_rate: 0.0024650435245075655
optimizer: sgd
Score: 0.7217630743980408


모든 측정지표에 직접 접근할 수도 있다.

In [96]:
best_trial.metrics.get_last_value("val_root_mean_squared_error")

0.7217630743980408

최상의 모델(best_model)의 성능이 만족스럽다면 전체 훈련세트에서 마저 더 훈련한 다음에 평가하고 배포하면 된다.

In [ ]:
norm_layer.adapt(X_train_full)#이게 빠져서 loss가 이상하게 계속 증가하고 있었네.
best_model.fit(X_train_full, y_train_full, epochs=10, verbose=2)
print(best_model.evaluate(X_test, y_test))

## 모델 자체가 아닌 그 외의 하이퍼 파라미터 튜닝

데이터 전처리 하이퍼파라미터나 또는 배치 크기 같은 model.fit()매개변수를 튜닝해야 할 수도 있다.

이런 경우에는 build_model()함수를 작성하는 대신에 kt.HyperModel클래스의 서브클래스를 만들고 build()와 fit()메서드 두개를 정의해야 한다.

build()메서드는 build_model()과 정확히 같은 역할을 한다.

fit()메서드는 HyperParameters객체와 컴파일된 모델, 그리고 model.fit()의 모든 매개변수를 인수로 받아서 모델을 훈련하고 History객체를 반환한다.
결정적으로 하이퍼파라미터를 사용해서 데이터 전처리 방법과 배치 크기 등을 결정할 수 있다.

In [110]:
class MyRegressionHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)

    def fit(self, hp, model, X, y, *args, **kwargs):
        if hp.Boolean("normalize"):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return    model.fit(X, y, *args, **kwargs)

그리고 build_model()함수 대신에 이 클래스의 객체를 원하는 튜너에 전달하면 된다.
예를 들어서 ky.HyperBand튜너를 만들어본다.

와 이거 실행하는데 13분 걸림 미친...

In [111]:
hyperband_tuner = kt.Hyperband(
    MyRegressionHyperModel(),
    objective="val_root_mean_squared_error",
    max_epochs=10,
    factor=3,#factor하고 hyperband_iterations얘네는 좀 복잡함.
    hyperband_iterations=2,
    directory="my_dir",
    project_name="my_model",
    overwrite=True
)

hyperband_tuner.search(
    X_train,
    y_train,
    epochs=10,
    validation_data=(X_valid, y_valid),
    verbose=2
)

Trial 60 Complete [00h 00m 44s]
val_root_mean_squared_error: 0.8854273557662964

Best val_root_mean_squared_error So Far: 0.709713339805603
Total elapsed time: 00h 13m 51s


In [127]:
best_hyperband_model = hyperband_tuner.get_best_models(num_models=1)[0]
best_hyperband_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 8)              │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 88)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            89 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,890 (187.07 KB)

 Trainable params: 47,873 (187.00 KB)

 Non-trainable params: 17 (72.00 B)

In [120]:
for hp in hyperband_tuner.get_best_hyperparameters(num_trials=3):
    print(hp.values)

{'n_hidden': 7, 'n_neurons': 88, 'learning_rate': 0.0013649679576430186, 'optimizer': 'sgd', 'normalize': False, 'tuner/epochs': 4, 'tuner/initial_epoch': 2, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0034'}
{'n_hidden': 5, 'n_neurons': 138, 'learning_rate': 0.0007730555977073046, 'optimizer': 'sgd', 'normalize': False, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0022'}
{'n_hidden': 5, 'n_neurons': 138, 'learning_rate': 0.0007730555977073046, 'optimizer': 'sgd', 'normalize': False, 'tuner/epochs': 4, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


In [128]:
norm_layer.adapt(X_train_full)
best_hyperband_model.fit(X_train_full, y_train_full, epochs=10, verbose=2)

print(best_hyperband_model.evaluate(X_test, y_test))

Epoch 1/10
484/484 - 2s - 5ms/step - loss: 0.6008 - root_mean_squared_error: 0.7751
Epoch 2/10
484/484 - 2s - 4ms/step - loss: 0.5336 - root_mean_squared_error: 0.7304
Epoch 3/10
484/484 - 2s - 4ms/step - loss: 0.8075 - root_mean_squared_error: 0.8986
Epoch 4/10
484/484 - 2s - 4ms/step - loss: 0.9852 - root_mean_squared_error: 0.9926
Epoch 5/10
484/484 - 2s - 4ms/step - loss: 0.5436 - root_mean_squared_error: 0.7373
Epoch 6/10
484/484 - 2s - 4ms/step - loss: 0.5333 - root_mean_squared_error: 0.7303
Epoch 7/10
484/484 - 2s - 4ms/step - loss: 0.5359 - root_mean_squared_error: 0.7321
Epoch 8/10
484/484 - 2s - 4ms/step - loss: 0.5326 - root_mean_squared_error: 0.7298
Epoch 9/10
484/484 - 2s - 4ms/step - loss: 0.6089 - root_mean_squared_error: 0.7803
Epoch 10/10
484/484 - 2s - 4ms/step - loss: 0.7520 - root_mean_squared_error: 0.8672
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5565 - root_mean_squared_error: 0.7459
[0.5537242889404297, 0.7441264986991882]
